## Imports


In [1]:
import os
import json
import pandas as pd
import psycopg2
from psycopg2 import OperationalError
from psycopg2 import ProgrammingError
import gc

## Data Connection


In [2]:
# read user file
with open("package/account/user.json", "r") as read_file:
    user = json.load(read_file)

In [3]:
# function to connect to postgresql db
def create_connection(db_name, db_user, db_password, db_host, db_port):
    try:
        connection = psycopg2.connect(
            database=db_name,
            user=db_user,
            password=db_password,
            host=db_host,
            port=db_port,
        )
        print("Connection to PostgreSQL DB successful")
    except OperationalError as e:
        connection = None
        print(f"The error '{e}' occurred")
        print("Error Type:", type(e))

    return connection

In [4]:
def read_secrets() -> dict:
    filename = os.path.join("secrets.json")
    try:
        with open(filename, mode="r") as f:
            return json.loads(f.read())
    except FileNotFoundError:
        return {}

In [5]:
secrets = read_secrets()


In [6]:
# set up database connection
con = create_connection(
    secrets["db_name"],
    secrets["db_user"],
    secrets["db_password"],
    secrets["db_host"],
    secrets["db_port"],
)


Connection to PostgreSQL DB successful


In [7]:
cur = con.cursor()


In [8]:
# function to select data, returns a dataframe
def select(query):
    data_df = None
    try:
        cur.execute(query)
        col_names = [desc[0] for desc in cur.description]
        data_df = pd.DataFrame(data=cur.fetchall(), columns=col_names)
    except ProgrammingError as e:
        con.rollback()
        print(f"The error '{e}' occurred")
        print("Error Type:", type(e))

    return data_df

## Import Data


In [9]:
select_query = """
SELECT *
FROM discord_data.dim_channel
"""

df_dim_channel = select(select_query)


In [10]:
select_query = """
SELECT *
FROM discord_data.dim_channel_type
"""

df_dim_channel_type = select(select_query)


In [11]:
select_query = """
SELECT *
FROM discord_data.dim_server
"""

df_dim_server = select(select_query)


In [12]:
select_query = """
SELECT *
FROM discord_data.fact_messages
"""

df_fact_messages = select(select_query)


In [13]:
del con
del cur
del read_file
del secrets
del select_query
del user
gc.collect()

0

In [14]:
# function to downcast all numeric columns and change all object columns to category
def downcast(input_df):
    print('Memory Usage before downcasting:')
    print(input_df.info(memory_usage='deep'))

    for column in input_df:
        if input_df[column].dtype == 'float64':
            input_df[column] = pd.to_numeric(
                input_df[column], downcast='float')
        if input_df[column].dtype == 'int64':
            input_df[column] = pd.to_numeric(
                input_df[column], downcast='integer')
        if input_df[column].dtype == 'object':
            input_df[column] = input_df[column].astype(
                'category')

    print('Memory Usage after downcasting:')
    print(input_df.info(memory_usage='deep'))

    return input_df

In [15]:
# read from csv (due to it being semi structured data)
# may transition to a NoSQL database in the future
# store each list as its own csv file
# length = 6
# df_fact_analytics = pd.DataFrame()
# for i in range(1, length+1, 1):
#     file_name = 'fact_analytics' + str(i) + '.csv'
#     df_temp = pd.read_csv(file_name)
#     df_fact_analytics = pd.concat([df_fact_analytics, df_temp], copy=False)
#     print(df_fact_analytics.shape)

# del df_temp

# not running due to memory limitations

In [16]:
# df_fact_analytics.head()

In [17]:
# read from csv (due to it being semi structured data)
# may transition to a NoSQL database in the future
df_fact_modeling = pd.read_csv("fact_modeling.csv")

C:\Users\Dante\AppData\Local\Temp\ipykernel_20716\1362443282.py:3: DtypeWarning: Columns (8,10,11,13,15,17,18,19,20,23,26,27,28,29,30,31,32,34,35,42,43,45,46,47,48,49,50,54,55,59,60,67,68,69,70,72,73,74,75,76,77,78,79,81,82,90,91,93,94,95,99,100,102,103,104,106,107,108,109,111,112,113,114,115,116,119,121,122,123,124,132,133,137,139,144,146,150,151,153,154,155,156,158,159,160,161,164,165,166,167,168,169,170,173,175,176,178,179,180,181,182,183,184,185,186,189,190,191,192,193,195,196,197,200,204,205,248,249,251,252,253,266,268,270,271,283,284,285,286,287,297,298,351,373,375,376,422,424,436,437,438,439,440,442,443,444,445,446,447,448,449,451,452,453,454,455,456,457,458,459,460,461,462,463,464,465,469,471,472,473,474,476,477,478,479,480,481,485,486,488,489,498,501,502,503,505,506,511,517,518,519,520,522,523,525,529,530,538,540,541,542,547,549,550,551,552,553,554,555,558,563,564,575,584,585,594,595,596,597,598,599,600,601,604,605,606,608,613,614,615,616,617,618,619,620,621,622,624,626,627,62

In [18]:
df_fact_modeling = downcast(df_fact_modeling)

Memory Usage before downcasting:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 884392 entries, 0 to 884391
Columns: 840 entries, event_type to is_premium_member
dtypes: float64(442), int64(2), object(396)
memory usage: 14.4 GB
None
Memory Usage after downcasting:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 884392 entries, 0 to 884391
Columns: 840 entries, event_type to is_premium_member
dtypes: category(396), float32(385), float64(57), int16(1), int64(1)
memory usage: 2.4 GB
None


In [19]:
df_fact_modeling.head()

,event_type,event_id,event_source,user_id,domain,freight_hostname,ip,day,chosen_locale,detected_locale,...,invite_channel_type,is_suggested,row_num,num_total,is_filtered,num_affinity_connections,send_type,role_subscription_group_listing_id,role_subscription_listing_ids,is_premium_member
0,open_popout,AQMETS3lsxQMTcnKBXSSIJuTXwAAARA=,client,342346882800025600,Modeling,analytics-ingest-prd-dnq6,14.192.209.0,1433,en-GB,en-US,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,open_popout,AQMETS3lsxQMTcnKBXSSIJuTXwAAAOg=,client,342346882800025600,Modeling,analytics-ingest-prd-50xt,14.192.209.0,1433,en-GB,en-US,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,open_popout,AQMEeInHNDc6ZoHEXkpHLouIvgAAACw=,client,342346882800025600,Modeling,analytics-ingest-prd-zjk4,103.253.105.0,1581,en-GB,en-US,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,open_popout,AQEENTa24ETKie63CPZELpoetk9NzOA=,client,342346882800025600,Modeling,analytics-ingest-prd-l5gj,175.143.61.0,1150,NaN,en-US,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,open_popout,AQEEDMhX5SkGx7oSxOeqENmYuGTYcZY=,client,342346882800025600,Modeling,analytics-ingest-prd-gqn5,14.192.212.0,1258,NaN,en-US,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
# read from csv (due to it being semi structured data)
# may transition to a NoSQL database in the future
df_fact_reporting = pd.read_csv("fact_reporting.csv")

C:\Users\Dante\AppData\Local\Temp\ipykernel_20716\473704480.py:3: DtypeWarning: Columns (10,11,13,14,16,27,28,34,36,40,41,43,44,45,46,47,50,51,52,59,60,63,65,67,68,69,70,71,76,77,82,84,90,91,93,94,95,96,97,98,99,100,101,102,103,104,106,107,108,120,121,124,125,126,128,129,130,131,132,133,134,135,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,153,155,157,158,159,160,164,165,168,169,172,173,175,176,178,191,192,194,195,196,198,200,209,210,241,249,251,252,253,254,261,262,263,264,273,274,275,276,277,278,279,281,283,284,285,286,287,288,292,294,295,304,305,306,307,308,309,312,313,314,315,316,317,318,319,320,321,325,326,329,333,335,336,338,339,342,343,344,347,369,370,372,373,374,378,380,381,384,385,390,391,392,393,395,397,398,399,400,401,402,403,405,406,407,408,409,410,411,412,413,414,416,417,418,419,422,423,430,431,434,435,436,438,439,444,448,450,451,453,454,455,457,460,461,462,463,470,471,472,473,474,475,477,479,480,481,482,483,485,487,488,489,497,499,500,501,502,505,508,509,510,

In [21]:
df_fact_reporting = downcast(df_fact_reporting)

Memory Usage before downcasting:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 631680 entries, 0 to 631679
Columns: 640 entries, event_type to num_applications_gog
dtypes: float64(303), int64(2), object(335)
memory usage: 8.5 GB
None
Memory Usage after downcasting:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 631680 entries, 0 to 631679
Columns: 640 entries, event_type to num_applications_gog
dtypes: category(335), float32(249), float64(54), int16(1), int64(1)
memory usage: 1.3 GB
None


In [22]:
df_fact_reporting.head()

,event_type,event_id,event_source,user_id,domain,freight_hostname,ip,day,chosen_locale,detected_locale,...,deeplink_source,link_type,num_applications_total,num_applications_battlenet,num_applications_discord,num_applications_steam,num_applications_twitch,num_applications_uplay,num_applications_origin,num_applications_gog
0,send_message,AQECpEkS/Zt1ZnmYLv6+1SQz+AAsKbk=,api,342346882800025600,Reporting,discord-api-7fc4cb59bd-fq45s,103.253.105.0,1855,en-GB,en-GB,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,send_message,AQECsEQ/SzYXFt3mJNphb896bABDpOU=,api,342346882800025600,Reporting,discord-api-8668c6c64f-28hwt,103.253.105.0,1786,en-GB,en-GB,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,send_message,AQECnoITHkGwoJeGW6v6cMtBcAAcd5I=,api,342346882800025600,Reporting,discord-api-8d47878f4-bz68m,103.253.105.0,1882,en-GB,en-GB,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,send_message,AQECoYnQsaKP11phtENl4D4TtwA4lk0=,api,342346882800025600,Reporting,discord-api-5bbbc7b7fd-j958j,103.253.105.0,1807,en-GB,en-GB,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,send_message,AQECsPMdnBCWr4xCKLo5tl1NnQAwif8=,api,342346882800025600,Reporting,discord-api-7fc4cb59bd-8wwk5,103.253.105.0,1855,en-GB,en-GB,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
# read from csv (due to it being semi structured data)
# may transition to a NoSQL database in the future
df_fact_tns = pd.read_csv("fact_tns.csv")

C:\Users\Dante\AppData\Local\Temp\ipykernel_20716\1577899063.py:3: DtypeWarning: Columns (7,9,10,11,12,13,14,15,16,18,19,20,21,22,23,24,25,26,27,28,29,32,35,41,42,44,45,46,55,64,65,68,69,70,72,74,75,76,77,78,79,80,81,83,84,85,100,101,103,104,105,124,125,135,136,137,148,149,223,224,226,227,249,250,251,253,288,299,300,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,326,327,331,337,339,343,344,346,347,348,349,351,352,353,354,357,358,359,360,361,362,363,365,369,370,371,372,374,375,376,377,378,379,383,393,394,395,396,397,398,399,400,401,410,411,413,416,417,419,421,422,423,426,427,428,430,431,432,440,441,444,448,454,456,457,459,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,479,480,481,482,485,486,493,495,496,497,498,500,501,502,503,504,505,506,507,508,509,510,511,512,513,517,519,520,521,522,524,525,526,527,528,529,533,534,543,545,546,580,588,590,591,592,598,601,602,604,611,612,615,617,618,621,622,623,624,626,631,632,633,634,635,636,63

In [24]:
df_fact_tns = downcast(df_fact_tns)

Memory Usage before downcasting:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 299924 entries, 0 to 299923
Columns: 1030 entries, event_type to is_greeting
dtypes: float64(571), int64(2), object(457)
memory usage: 5.8 GB
None
Memory Usage after downcasting:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 299924 entries, 0 to 299923
Columns: 1030 entries, event_type to is_greeting
dtypes: category(457), float32(511), float64(60), int16(1), int64(1)
memory usage: 1.0 GB
None


In [25]:
df_fact_tns.head()

,event_type,event_id,event_source,user_id,domain,freight_hostname,freight_id,ip,day,chosen_locale,...,account_id,account_name,connected,visibility,friend_sync,partner,link_method,two_way_link,metadata_visibility,is_greeting
0,user_phone_updated,AQEFOgYw9XUViwEa4YXTjqFZwAEaNDQ=,api,342346882800025600,Tns,discord-api-9869d5fcc-lkwnc,UZh5EmxHMAJjj_kWNDQaAQ==,103.253.105.84,1781,en-GB,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,user_phone_updated,AQEFh45YBraYxFbvwxkzv1TgLgAHCww=,api,342346882800025600,Tns,discord-api-5695f55cb5-xqq4x,Gze9hEhXDjDyhwMXDAsHAA==,103.253.105.84,1812,en-GB,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,guild_viewed,AQMFeYhBPLDjh+cE4sxgHk5ipwAABAM=,client,342346882800025600,Tns,analytics-ingest-prd-q0pz,HiRxikGPr_LjE-4W8dk0ig==,103.253.105.84,1757,en-GB,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,guild_viewed,AQMFeYhBPLDjh+cE4sxgHk5ipwAABEs=,client,342346882800025600,Tns,analytics-ingest-prd-424c,V-dmfzra_D09FO4Wobpeiw==,103.253.105.84,1757,en-GB,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,guild_viewed,AQMFeYhBPLDjh+cE4sxgHk5ipwAAA1A=,client,342346882800025600,Tns,analytics-ingest-prd-h4p0,26SaCyp1G_3jE-4W3ggrWA==,103.253.105.84,1756,en-GB,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
%store df_dim_channel
%store df_dim_channel_type
%store df_dim_server
%store df_fact_messages
# %store df_fact_analytics
%store df_fact_modeling
%store df_fact_reporting
%store df_fact_tns


Stored 'df_dim_channel' (DataFrame)
Stored 'df_dim_channel_type' (DataFrame)
Stored 'df_dim_server' (DataFrame)
Stored 'df_fact_messages' (DataFrame)
Stored 'df_fact_modeling' (DataFrame)
Stored 'df_fact_reporting' (DataFrame)
Stored 'df_fact_tns' (DataFrame)


## Analysis


In [27]:
%store -r

## Ideas

- Check readability of text (texttatistic)
-